 # K-nearest-neighbours (KNN)
## Etapes:
#### 1. Chargement des  données
- Charger les images et les étiquettes (malade ou non) à partir des dossiers de données.

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np

from src.preprocessing import load_images_from_folder
import os

base_path = os.path.abspath(os.path.join(os.getcwd(), "../../data/raw_test"))

train_path = os.path.join(base_path, "train")

X, y = load_images_from_folder(train_path, image_size=(64, 64))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'C:\\Users\\ramda\\PycharmProjects\\T-DEV-810-PAR_8\\data\\raw_test\\train'


#### 2.Normalisation et prétraitement
- Ajuster les valeurs des pixels pour qu'elles soient dans une plage spécifique (souvent 0 à 1)
- Transformer chaque image 2D (64x64 pixels) en un vecteur 1D (4096 éléments.

In [21]:

from src.preprocessing import normalize_images
import numpy as np

normalized_images = normalize_images(X_train)
flattened_images = np.array([x.flatten() for x in normalized_images])

test_normalized_images = normalize_images(X_test)
test_flattened_images = np.array([x.flatten() for x in test_normalized_images])

#### 3. Définition des hyperparamètres et validation croisée : 
- On définit une  grille d'hyperparamètres pour tester différents nombres de voisins (k).
- Utiliser GridSearchCV pour trouver le meilleur hyperparamètre n_neighbors avec validation croisée.


In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np

param_grid = {'n_neighbors': np.arange(1, 31)}

knn_gscv = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)


#### 4.Récupération du Meilleur Nombre de Voisins
- Entraîner le modèle KNN avec les données d'entraînement après les avoir aplaties.
- Identifier le meilleur nombre de voisins (k) trouvé par GridSearchCV.

In [15]:
# Entraînement avec les données d'entraînement aplaties
knn_gscv.fit(flattened_images, y_train)

# Récupération du meilleur nombre de voisins et du meilleur score
best_k = knn_gscv.best_params_['n_neighbors']
best_score = knn_gscv.best_score_
print(f'Best number of neighbors: {best_k}')
print(f'Best cross-validated score: {best_score:.4f}')  

NameError: name 'knn_gscv' is not defined

#### 5.Entraînement du modèle avec le meilleur k
- Créer un nouveau modèle KNN avec le meilleur n_neighbors trouvé et l'entraîner sur les données d'entraînement.

In [ ]:
# Entraînement du modèle avec le meilleur K
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(flattened_images, y_train)

#### 6.Évaluation sur l'ensemble de test
- Évaluer les performances du modèle KNN optimal sur un ensemble de test indépendant.

In [16]:
y_test_pred = knn_best.predict(test_flattened_images)
print(classification_report(y_test, y_test_pred))
print("confusion matrix: \n",confusion_matrix(y_test, y_test_pred))

NameError: name 'knn_best' is not defined

In [17]:
# Calcul des probabilités prédictives pour la classe positive
y_test_pred_proba = knn_best.predict_proba(test_flattened_images)[:, 1]

# Calcul des taux de faux positifs et de vrais positifs
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)

# Calcul de l'AUC
auc = roc_auc_score(y_test, y_test_pred_proba)

# Affichage de la courbe ROC
plt.figure()
plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

NameError: name 'knn_best' is not defined